# Descriptive Spatial Analysis

#### !!! As this notebook uses h3 package it will NOT WORK ON WINDOWS !!!

This notebook reads in hexagon files and does a desprictive spatial analysis.

### What is done?
1. Load Hexagon files
2. TODO

### Input

- data/output/Trips_Hexagons.csv

### Output
- Nothing, only plots
    
---------------------------------------------------------

In [ ]:
import os
import pandas as pd
from h3 import h3
from shapely.geometry import shape
import geopandas
import matplotlib.pyplot as plt
import folium
from folium.plugins import TimeSliderChoropleth

In [ ]:
path_trips_hex=os.path.join(os.getcwd(), "..", "data", "output", "Trips.csv")
path_hex_small=os.path.join(os.getcwd(), "..", "data", "output", "hexagons-small-dresden.geojson")
path_hex_big=os.path.join(os.getcwd(), "..", "data", "output", "hexagons-big-dresden.geojson")

print(path_trips_hex)
print(path_hex_small)
print(path_hex_big)

In [ ]:
df_trips_hex = pd.read_csv(path_trips_hex, index_col="Unnamed: 0")
df_trips_hex

In [ ]:
gdf_hex_small = geopandas.read_file(path_hex_small)
# gdf_hex_small.rename({"h3_hex_sma": "h3_hex_small_id"}, inplace=True, axis=1)
gdf_hex_small.plot()

gdf_hex_big = geopandas.read_file(path_hex_big)
# gdf_hex_big.rename({"h3_hex_big": "h3_hex_big_id"}, inplace=True, axis=1)
gdf_hex_big.plot()

display(gdf_hex_small, gdf_hex_big)

## 2. Create GeoDataFrames

In [ ]:
# clean csv data
df_trips_start_small = df_trips_hex[["h3_hex_small_id_start", "24_sum"]]
df_trips_start_small = df_trips_start_small.groupby("h3_hex_small_id_start").agg({"24_sum": "sum"})
display(df_trips_start_small)

# join the geodataframe with the cleaned up csv dataframe
# df_trips_start_small = df_trips_hex.drop(["h3_hex_small_id_end", "h3_hex_big_id_start", "h3_hex_big_id_end"], axis=1)
df_merged_small = gdf_hex_small.set_index('h3_hex_small_id').join(df_trips_start_small)
# df_merged_small.dropna(inplace=True)
# df_merged_small.head()
df_merged_small

In [ ]:
gdf_hex_small = geopandas.read_file(path_hex_small)
gdf_hex_big = geopandas.read_file(path_hex_big)
display(gdf_hex_small, gdf_hex_big)

In [ ]:
"""
df_copy = df_trips_hex.copy()
df_copy["datetime_start"] = (pd.to_datetime(df_trips_hex["datetime_start"]).astype(int)//10**9)
styledict_timeslider={}
for i in df_copy['h3_hex_small_id_start'].unique():
    styledict_timeslider[i]={}
    for j in df_copy[df_copy['h3_hex_small_id_start']==i].set_index(['h3_hex_small_id_start']).values:
        print(j)
        styledict_timeslider[i][j[0]]={'color':j[],'opacity':0.7}
        
styledict_timeslider
"""

In [ ]:
def plot_choropleth_map(df):
    """Plots a choropleth graph on a map based on the number of started trips in each hexagon.
    The fixed bike stations are marked as well
    
    Args:
        p_df (DataFrame):   DataFrame with trip data from nuremberg
        p_mode (str):       String which contains mode in which program is running
    
    Returns:
        no return
    """
    # finding the month with most trips in the month
    month_most = df.groupby(by="month_start").count().idxmax()["datetime_start"]
    print("Month with most rides:", month_most)

    df_biggest_month = df[df["month_start"] == month_most][["h3_hex_small_id_start", "h3_hex_big_id_start", "24_sum"]]
    # prints the number of trips per postal code code
    df_map = df[["h3_hex_small_id_start", "h3_hex_big_id_start", "booking_date_start"]].copy()
    # df_map_small.loc[:, "count"] = 0
    df_map_small = df_map.groupby(by=["booking_date_start", "h3_hex_small_id_start"]).count().reset_index()
    df_map_small.rename(columns={"h3_hex_small_id_start": "hex_id", "h3_hex_big_id_start": "count"}, inplace=True)
    # TODO: TAKE ALL DAYS IN ACCOUNT WITH TIME SLIDER
    df_map_small = df_map_small[df_map_small["booking_date_start"] == "2019-01-20 00:00:00"]
    print("Max count:", df_map_small["count"].max(), "// Min count:", df_map_small["count"].min())
    display(df_map_small)
    
    df_map_big = df_map.groupby(by=["booking_date_start", "h3_hex_big_id_start"]).count().reset_index()
    df_map_big.rename(columns={"h3_hex_big_id_start": "hex_id", "h3_hex_small_id_start": "count"}, inplace=True)
    # TODO: TAKE ALL DAYS IN ACCOUNT WITH TIME SLIDER
    df_map_big = df_map_big[df_map_big["booking_date_start"] == "2019-01-20 00:00:00"]
    print("Max count:", df_map_big["count"].max(), "// Min count:", df_map_big["count"].min())
    display(df_map_big)
    # df_map_big = df_biggest_month.groupby(by="h3_hex_big_id_start")["24_sum"].count().reset_index()

    display(df_map_big)
    
    m = folium.Map([51.08, 13.74], zoom_start=12, keep_in_front=True)

    folium.Choropleth(
        geo_data=gdf_hex_small,
        name="Demand per Small Hexagon (Choropleth)",
        data=df_map_small,
        columns=["hex_id", "count"],
        key_on="feature.properties.h3_hex_small_id",
        legend_name="Trips per small hexagon",
        fill_color="Reds",
        fill_opacity=0.7,
        line_opacity=0.5
    ).add_to(m)
    
    folium.Choropleth(
        geo_data=gdf_hex_big,
        name="Demand per Big Hexagon (Choropleth)",
        data=df_map_big,
        columns=["hex_id", "count"],
        key_on="feature.properties.h3_hex_big_id",
        legend_name="Trips per big hexagon",
        fill_color="Blues",
        fill_opacity=0.7,
        line_opacity=0.5,
        show=False
    ).add_to(m)

    df = df[df["p_number_start"] != 0.0]
    # p_df = p_df[p_df["End_Place_id"] != 0.0]
    df_stations = df.drop_duplicates("p_number_start", keep="first")
    # df_stations = p_df.drop_duplicates("End_Place_id", keep="first")
    
    fixedStations = folium.FeatureGroup(name="Fixed Stations")
    
    for index, row in df_stations.iterrows():
        folium.CircleMarker(
            [row["p_lat_start"], row["p_lng_start"]],
            radius=3,
            popup=folium.Popup(
                "<b>Station Name:</b><br>" +
                str(row["p_name_start"]), max_width=400),
            fill=True,
            fill_color="#3db7e4",
            color="#3db7e4"
        ).add_to(fixedStations)

    fixedStations.add_to(m)
    m.keep_in_front(fixedStations)
        
    folium.LayerControl().add_to(m)
    
    # TimeSliderChoropleth(
    #     data=gdf_hex_small,
    #     name="Hexagons over time",
    #     styledict=styledict_timeslider
    # ).add_to(m)

    uni = [51.0292, 13.729]
    folium.Marker(
        location=uni,
        tooltip="University",
        icon=folium.Icon(color="black", icon="home"),
    ).add_to(m)

    path_map=os.path.join(os.getcwd(), "..", "data", "output", "choropleth_map.html")
    
    m.save(path_map)

In [ ]:
plot_choropleth_map(df_trips_hex)

### 2.1. Define common method

In [ ]:
"""
def createGeoDataFrameWithGeometryOfPosition(position, hex_size):
    # Create geo data frame
    gdf_trips_hex = geopandas.GeoDataFrame(df_trips_hex)
    print("DONE Creating GeoDataFrame")
    
    # dynamically create name of column to transform into geometry
    # e.g.: if position=start and hex_size=small, builds: h3_hex_small_id_start
    geometryColumn = 'h3_hex_' + hex_size + '_id_' + position
    gdf_trips_hex['geometry'] = gdf_trips_hex[(gdf_trips_hex[geometryColumn].notna())].apply(lambda row: shape({
                                            "type": "Polygon",
                                            "coordinates": [h3.h3_to_geo_boundary(row[geometryColumn], geo_json=True)],
                                            "properties": ""
                                    }), axis=1)
    gdf_trips_hex = gdf_trips_hex.set_crs('EPSG:4326')
    print("DONE Building Geometry for: Small Hex / Start")
    return gdf_trips_hex
    """

### 2.2. GeoDataFrame for START position and SMALL hexagons

In [ ]:
# gdf_trips_hex_small_start = createGeoDataFrameWithGeometryOfPosition(position="start", hex_size="small")
# gdf_trips_hex_small_start.plot()
# gdf_trips_hex_small_start

### 2.3. GeoDataFrame for END position and SMALL hexagons

In [ ]:
# gdf_trips_hex_small_end = createGeoDataFrameWithGeometryOfPosition(position="end", hex_size="small")
# gdf_trips_hex_small_end.plot()
# gdf_trips_hex_small_end

### 2.4. GeoDataFrame for START position and BIG hexagons

In [ ]:
# gdf_trips_hex_big_start = createGeoDataFrameWithGeometryOfPosition(position="start", hex_size="big")
# gdf_trips_hex_big_start.plot()
# gdf_trips_hex_big_start

### 2.5. GeoDataFrame for END position and BIG hexagons

In [ ]:
# gdf_trips_hex_big_end = createGeoDataFrameWithGeometryOfPosition(position="end", hex_size="big")
# gdf_trips_hex_big_end.plot()
# gdf_trips_hex_big_end

## 3. Plot Choropleth maps

In [ ]:
# color_scale = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], 
#                [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

# data = [ dict(
#             type='choropleth', # type of map-plot
#             colorscale = color_scale,
#             autocolorscale = False,
#             locations = gdf_trips_hex_small_start['h3_hex_small_id_start'], # the column with the hex id
#             z = df_['Murder_per100000'].astype(float), # the variable I want to color-code
#             locationmode = 'USA-states',
#             text = df_sected_crime['text'], # hover text
#             marker = dict(     # for the lines separating states
#                         line = dict (
#                                   color = 'rgb(255,255,255)', 
#                                   width = 2) ),               
#             colorbar = dict(
#                         title = "Murder rate per 100,000 people")
#             ) 
#        ]